<a href="https://colab.research.google.com/github/FinnRobertson15/PatternAnalysis-2023/blob/topic-recognition/Colab%20version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive

[Errno 2] No such file or directory: '/content/drive/MyDrive/Root'
/content/drive/MyDrive


In [ ]:
!unzip ADNI_AD_NC_2D.zip

Streaming output truncated to the last 5000 lines.
  inflating: AD_NC/test/AD/413796_78.jpeg  
  inflating: AD_NC/test/AD/413796_79.jpeg  
  inflating: AD_NC/test/AD/413796_80.jpeg  
  inflating: AD_NC/test/AD/413796_81.jpeg  
  inflating: AD_NC/test/AD/413796_82.jpeg  
  inflating: AD_NC/test/AD/413796_83.jpeg  
  inflating: AD_NC/test/AD/413796_84.jpeg  
  inflating: AD_NC/test/AD/413796_85.jpeg  
  inflating: AD_NC/test/AD/413796_86.jpeg  
  inflating: AD_NC/test/AD/413796_87.jpeg  
  inflating: AD_NC/test/AD/413796_88.jpeg  
  inflating: AD_NC/test/AD/413796_89.jpeg  
  inflating: AD_NC/test/AD/413796_90.jpeg  
  inflating: AD_NC/test/AD/413796_91.jpeg  
  inflating: AD_NC/test/AD/413796_92.jpeg  
  inflating: AD_NC/test/AD/413796_93.jpeg  
  inflating: AD_NC/test/AD/413796_94.jpeg  
  inflating: AD_NC/test/AD/413796_95.jpeg  
  inflating: AD_NC/test/AD/413796_96.jpeg  
  inflating: AD_NC/test/AD/413796_97.jpeg  
  inflating: AD_NC/test/AD/414389_78.jpeg  
  inflating: AD_NC/test/A

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import time
import numpy as np
# import pandas as pd
import math
import os
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import ConcatDataset

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_paths[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image

class CustomConcatDataset(ConcatDataset):
    def __init__(self, datasets, labels):
        super(CustomConcatDataset, self).__init__(datasets)
        self.labels = labels

    def __getitem__(self, index):
        item = super(CustomConcatDataset, self).__getitem__(index)
        return item, self.labels[index]

def combine(AD, NC):
    # Set labels for the AD dataset
    labels_AD = [True] * len(AD)
    # Set labels for the NC dataset
    labels_NC = [False] * len(NC)

    # Combine the datasets and labels
    X = AD + NC
    Y = labels_AD + labels_NC
    # X = torch.utils.data.RandomSampler(X)
    seed = 123
    torch.manual_seed(seed)
    np.random.seed(seed)


    X_loader = DataLoader(X, batch_size=128, shuffle=True)
    Y_loader = DataLoader(Y, batch_size=128, shuffle=True)
    return X_loader, Y_loader

size = 256

transform_X = transforms.Compose([
    transforms.Resize((size, size)),  # Resize the image to the desired size
    transforms.ToTensor(),         # Convert the image to a tensor
    transforms.Lambda(lambda x: x / 255.0),
    transforms.Lambda(lambda x: (x - x.mean()) / x.std())  # Subtract mean and divide by standard deviation
])



# Replace 'your_nii_folder' with the path to your folder containing .nii files
# nii_dataset = NiiDataset(root_dir=r'keras_png_slices_data\keras_png_slices_data\keras_png_slices_train')
loaders = {}

for stage in ['train', 'test']:
    loaders[stage] = {}
    AD = CustomDataset(root_dir=rf'ADNI_AD_NC_2D\AD_NC\{stage}\AD', transform=transform_X)
    NC = CustomDataset(root_dir=rf'ADNI_AD_NC_2D\AD_NC\{stage}\NC', transform=transform_X)
    loaders[stage]['X'], loaders[stage]['Y'] = combine(AD, NC)
# Y_train  = CustomDataset(root_dir=r'ADNI_AD_NC_2D\AD_NC\train\NC', transform=transform_Y)
# X_train_AD_loader = DataLoader(X_train_AD, batch_size=4, shuffle=True)
# Y_train_loader = DataLoader(Y_train, batch_size=4, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.maxpool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.maxpool2 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.softmax(self.fc2(x), dim=1)
        return x



# Instantiate the model
model = CustomModel()



In [ ]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
total_step = len(loaders['train']['X'])
sched_linear_1 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.005, max_lr=learning_rate, step_size_up=15, step_size_down=15, mode="triangular", verbose=False)
sched_linear_3 = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.005/learning_rate, end_factor=0.005/learning_rate, verbose=False)
scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[sched_linear_1, sched_linear_3], milestones=[30])


In [ ]:
num_epochs = 2
for epoch in range(num_epochs):

  for i, (images, seg) in enumerate(zip(loaders['train']['X'], loaders['train']['Y'])):
    images = images.to(device)
    seg = torch.tensor(seg, dtype=torch.long).to(device)
    outputs = model(images)


    # Adjust the shapes to match the criterion
    # outputs = outputs.view(-1)  # Flatten the outputs to a 1D tensor
    # seg = seg.view(-1)  # Flatten the seg tensor to a 1D tensor
    loss = criterion(outputs, seg)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 5 == 0:
      print(f"Epoch [{epoch + 1} / {num_epochs}], Step [{i + 1} / {total_step} Loss {loss.item()}]")

  scheduler.step()
end = time.time()